In [2]:
import torch
import sys
import pickle
sys.path.append('../../data_processing')
sys.path.append('../../model')
sys.path.append('../../../data')
import CNN, CNN_multimodal, CNN_LSTM
import TF_data_loader
from torch.utils import data as D
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
torch.set_printoptions(threshold=torch.inf)
np.set_printoptions(threshold=np.inf)

In [4]:
YY1_sorted_sequence_score_topten = torch.load('YY1_sorted_sequence_score_topten.pt')
other_sorted_sequence_score_topten = torch.load('other_sorted_sequence_score_topten.pt')

In [5]:
feat_map = torch.load('../../../data/Chipseq_data/FeatMap.pt')
feat_map = {v: k for k, v in feat_map.items()}

In [6]:
def sequence_union(data):
    result = []
    for group in data:
        for row in group:
            cell_line = row[0]
            saliency = row[1]
            predicted_score = row[2]
            null_predicted_score = row[3]
            sequence = row[4]
            feature_vector = row[5]
            found = False
            for i, r in enumerate(result):
                if r[4] == sequence:
                    result[i][0] += ';' + str(cell_line)
                    result[i][1] += ';' + str(saliency)
                    result[i][2] += ';' + str(predicted_score)
                    result[i][3] += ';' + str(null_predicted_score)
                    found = True
                    break
            if not found:
                result.append([str(cell_line), str(saliency), str(predicted_score), str(null_predicted_score), sequence, feature_vector])
    return result

In [7]:
def sequence_comparison(YY1, other):
    result = []
    for row in YY1:
        cell_line = row[0]
        saliency = row[1]
        predicted_score = row[2]
        null_predicted_score = row[3]
        sequence = row[4]
        feature_vector = row[5]
        intersections = []
        for group in other:
            for r in group:
                if r[4] == sequence:
                    if r[0] not in intersections:
                        intersections.append(r[0])
        formatted_intersections = ""
        for element in intersections:
            if formatted_intersections == "":
                formatted_intersections += element
            else:
                formatted_intersections += ';' + element
        result.append([cell_line, saliency, predicted_score, null_predicted_score, sequence, feature_vector, formatted_intersections])
    return result

In [8]:
union_set = sequence_union(YY1_sorted_sequence_score_topten)

In [9]:
megaset = sequence_comparison(union_set, other_sorted_sequence_score_topten)

In [10]:
df = pd.DataFrame(megaset, columns=["Cell Line","Score Difference","Predicted Score","Null Predicted Score","Sequence","Feature Vector","Intersections"])
df.to_csv('YY1 common sequences with intersections.tsv', sep='\t', index=False)